## python code to mix noise with audio / speech data

### import libraries

In [ ]:
import math   
import numpy as np 
import librosa
import matplotlib.pyplot as plt

### 2 ways of generating noise
- calculate Additive White Gaussian Noise (AWGN) from given audio
- extract and add noise from a noisy wav file

In [ ]:
#SNR in dB
#given a signal and desired SNR, this gives the required AWGN what should be added to the signal to get the desired SNR
def get_white_noise(signal,SNR) :
    #RMS value of signal
    RMS_s=math.sqrt(np.mean(signal**2))
    #RMS values of noise
    RMS_n=math.sqrt(RMS_s**2/(pow(10,SNR/10)))
    #Additive white gausian noise. Thereore mean=0
    #Because sample length is large (typically > 40000)
    #we can use the population formula for standard daviation.
    #because mean=0 STD=RMS
    STD_n=RMS_n
    noise=np.random.normal(0, STD_n, signal.shape[0])
    return noise

#given a signal, noise (audio) and desired SNR, this gives the noise (scaled version of noise input) that gives the desired SNR
def get_noise_from_sound(signal,noise,SNR):
    RMS_s=math.sqrt(np.mean(signal**2))
    #required RMS of noise
    RMS_n=math.sqrt(RMS_s**2/(pow(10,SNR/10)))
    
    #current RMS of noise
    RMS_n_current=math.sqrt(np.mean(noise**2))
    noise=noise*(RMS_n/RMS_n_current)
    
    return noise


In [ ]:
#***convert complex np array to polar arrays (2 apprays; abs and angle)
def to_polar(complex_ar):
    return np.abs(complex_ar),np.angle(complex_ar)

### method1: add additive white gaussian noise

In [ ]:
#
#*add AWGN noise
#
signal_file='20210223_arne_grind.wav'
signal, sr = librosa.load(signal_file)
signal=np.interp(signal, (signal.min(), signal.max()), (-1, 1))
noise=get_white_noise(signal,SNR=10)
#analyze the frequency components in the signal
X=np.fft.rfft(noise)
radius,angle=to_polar(X)
plt.plot(radius)
plt.xlabel("FFT coefficient")
plt.ylabel("Magnitude")
plt.show()
signal_noise=signal+noise
plt.plot(signal_noise)
plt.xlabel("Sample number")
plt.ylabel("Amplitude")
plt.show()


### method2: extract and add noise from a wav file 

In [ ]:
# helper script to crop the noise file to match audio file
noise_file='3S0D_12_8_0_16_0_0_0.wav'
noise, sr = librosa.load(noise_file)

if (len(noise)<len(signal)):
    while (len(noise)<len(signal)):
        noise = np.append(noise, noise)
        noise=noise[0:len(signal)]
elif(len(noise)>len(signal)):
    noise=noise[0:len(signal)]

assert len(noise) == len(signal)

In [ ]:
#*
#*add real world noise*
#*
signal, sr = librosa.load(signal_file)
signal=np.interp(signal, (signal.min(), signal.max()), (-1, 1))
plt.plot(signal)
plt.xlabel("Sample number")
plt.ylabel("Signal amplitude")
plt.show()

noise_file='3S0D_12_8_0_16_0_0_0.wav'
noise, sr = librosa.load(noise_file)
noise=np.interp(noise, (noise.min(), noise.max()), (-1, 1))
noise=get_noise_from_sound(signal,noise,SNR=1)

signal_noise=signal+noise

print("SNR = " + str(20*np.log10(math.sqrt(np.mean(signal**2))/math.sqrt(np.mean(noise**2)))))

plt.plot(signal_noise)
plt.xlabel("Sample number")
plt.ylabel("Amplitude")
plt.show()

### save the noisy audio file

In [ ]:
from scipy.io.wavfile import write
write("noisy_speech.wav",sr,signal_noise)